In [ ]:
import requests

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=RELIANCE.BSE&outputsize=full&apikey=SDJSQFL5SBTZZA55'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
import requests
import pandas as pd

def fetch_intraday_data(symbol, interval, api_key):
    """
    Fetches intraday data for a given stock symbol and interval using Alpha Vantage API.

    Args:
        symbol (str): Stock symbol (e.g., "AAPL").
        interval (str): Time interval (e.g., "1min", "5min").
        api_key (str): Alpha Vantage API key.

    Returns:
        pd.DataFrame: DataFrame containing intraday stock data.
    """
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": interval,
        "apikey": api_key,
        "datatype": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Parse the 'Time Series' data
    time_series_key = f"Time Series ({interval})"
    if time_series_key in data:
        df = pd.DataFrame.from_dict(data[time_series_key], orient="index")
        df.columns = [col.split(". ")[1] for col in df.columns]
        df.index = pd.to_datetime(df.index)
        return df
    else:
        print("Error:", data.get("Note") or data.get("Error Message"))
        return pd.DataFrame()

# Example usage
API_KEY = "your_alpha_vantage_api_key"
symbol = "AAPL"
interval = "1min"
intraday_data = fetch_intraday_data(symbol, interval, API_KEY)
print(intraday_data.head())


In [ ]:
import requests
import pandas as pd

def fetch_daily_data(symbol, api_key, outputsize="compact"):
    """
    Fetches daily historical data for a given stock symbol using Alpha Vantage API.

    Args:
        symbol (str): Stock symbol (e.g., "GOOGL").
        api_key (str): Alpha Vantage API key.
        outputsize (str): Data size ('compact' for 100 entries, 'full' for entire history).

    Returns:
        pd.DataFrame: DataFrame containing daily stock data.
    """
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": symbol,
        "outputsize": outputsize,
        "apikey": api_key,
        "datatype": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Parse the 'Time Series' data
    if "Time Series (Daily)" in data:
        df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")
        df.columns = [col.split(". ")[1] for col in df.columns]
        df.index = pd.to_datetime(df.index)
        return df
    else:
        print("Error:", data.get("Note") or data.get("Error Message"))
        return pd.DataFrame()

# Example usage
API_KEY = "your_alpha_vantage_api_key"
symbol = "GOOGL"
daily_data = fetch_daily_data(symbol, API_KEY, outputsize="full")
print(daily_data.head())


In [ ]:
import requests
import pandas as pd

def fetch_weekly_data(symbol, api_key):
    """
    Fetches weekly historical stock data using the Alpha Vantage API.

    Args:
        symbol (str): Stock ticker symbol (e.g., "MSFT").
        api_key (str): Alpha Vantage API key.

    Returns:
        pd.DataFrame: Weekly stock data with open, high, low, close, and volume.
    """
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_WEEKLY",
        "symbol": symbol,
        "apikey": api_key,
        "datatype": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Parse the 'Weekly Time Series' data
    if "Weekly Time Series" in data:
        df = pd.DataFrame.from_dict(data["Weekly Time Series"], orient="index")
        df.columns = [col.split(". ")[1] for col in df.columns]  # Remove prefixes
        df.index = pd.to_datetime(df.index)
        df = df.sort_index(ascending=True)  # Ensure the data is in chronological order
        df = df.astype(float)  # Convert data to numeric types for analysis
        return df
    else:
        print("Error:", data.get("Note") or data.get("Error Message"))
        return pd.DataFrame()

# Example usage
API_KEY = "your_alpha_vantage_api_key"
symbol = "MSFT"
weekly_data = fetch_weekly_data(symbol, API_KEY)

# Display and save the data
print(weekly_data.head())
weekly_data.to_csv("weekly_stock_data.csv")


In [ ]:
url = f"https://finnhub.io/api/v1/quote?symbol={symbol}&token={API}"
response = requests.get(url)
print(response.json())

In [ ]:
import time
start_date = int(time.mktime(time.strptime("2024-01-01", "%Y-%m-%d")))
end_date = int(time.mktime(time.strptime("2024-02-01", "%Y-%m-%d")))


In [ ]:
import requests
import pandas as pd

def fetch_finnhub_data(symbol, api_key, resolution='D', start_date='1704067200', end_date='1706659200'):
    """
    Fetches historical stock price data from Finnhub API.

    Args:
        symbol (str): Stock ticker symbol (e.g., "AAPL").
        api_key (str): Finnhub API key.
        resolution (str): Time resolution ('1', '5', '15', '30', '60', 'D', 'W', 'M').
        start_date (str): Start date (UNIX timestamp).
        end_date (str): End date (UNIX timestamp).

    Returns:
        pd.DataFrame: DataFrame containing stock prices.
    """
    url = f"https://finnhub.io/api/v1/stock/candle"
    params = {
        'symbol': symbol,
        'resolution': resolution,
        'from': start_date,  # Example: UNIX timestamp
        'to': end_date,      # Example: UNIX timestamp
        'token': api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data.get('s') != 'ok':
        print("Error fetching data:", data.get('error'))
        return pd.DataFrame()

    # Convert to DataFrame
    df = pd.DataFrame({
        'date': pd.to_datetime(data['t'], unit='s'),
        'open': data['o'],
        'high': data['h'],
        'low': data['l'],
        'close': data['c'],
        'volume': data['v']
    })

    return df

# Example usage
API_KEY = 'your_finnhub_api_key'  # Replace with your API key
symbol = 'AAPL'  # Apple Inc.
df = fetch_finnhub_data(AAPL, API_KEY)

if not df.empty:
    print(df.head())
    df.to_csv('finnhub_stock_data.csv', index=False)
    print("Data saved to finnhub_stock_data.csv")
else:
    print("No data retrieved.")


In [ ]:
import requests
import pandas as pd
import time

def get_finnhub_stock_data(symbol, resolution, start_date, end_date, api_key):
    """
    Fetch stock price data from Finnhub API.

    Args:
        symbol (str): Stock ticker symbol (e.g., "AAPL").
        resolution (str): Time resolution ('D' for daily, 'W' for weekly).
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        api_key (str): Finnhub API key.

    Returns:
        pd.DataFrame: DataFrame containing stock data.
    """

    # Convert dates to UNIX timestamps
    start_timestamp = int(time.mktime(time.strptime(start_date, "%Y-%m-%d")))
    end_timestamp = int(time.mktime(time.strptime(end_date, "%Y-%m-%d")))

    url = "https://finnhub.io/api/v1/stock/candle"
    params = {
        'symbol': symbol,
        'resolution': resolution,  # 'D' for daily, 'W' for weekly
        'from': start_timestamp,
        'to': end_timestamp,
        'token': api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data.get('s') != 'ok':
        print("Error fetching data:", data.get('error'))
        return pd.DataFrame()

    # Convert to DataFrame
    df = pd.DataFrame({
        'date': pd.to_datetime(data['t'], unit='s'),
        'open': data['o'],
        'high': data['h'],
        'low': data['l'],
        'close': data['c'],
        'volume': data['v']
    })

    return df

# Example usage
API_KEY = 'your_finnhub_api_key'  # Replace with your API key
symbol = 'AAPL'  # Stock symbol for Apple Inc.
start_date = '2024-01-01'
end_date = '2024-01-31'

# Fetch daily data
daily_data = get_finnhub_stock_data(symbol, 'D', start_date, end_date, API_KEY)
print("Daily Data:\n", daily_data.head())
daily_data.to_csv('finnhub_daily_data.csv', index=False)

# Fetch weekly data
weekly_data = get_finnhub_stock_data(symbol, 'W', start_date, end_date, API_KEY)
print("Weekly Data:\n", weekly_data.head())
weekly_data.to_csv('finnhub_weekly_data.csv', index=False)
